In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import Sequence

In [5]:
# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/data_balita.csv")

Mounted at /content/drive


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120999 entries, 0 to 120998
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Umur (bulan)       120999 non-null  int64  
 1   Jenis Kelamin      120999 non-null  object 
 2   Tinggi Badan (cm)  120999 non-null  float64
 3   Status Gizi        120999 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.7+ MB


In [7]:
data.head(15)

Umur (bulan) Jenis Kelamin  Tinggi Badan (cm)       Status Gizi
0              0     laki-laki          44.591973           stunted
1              0     laki-laki          56.705203            tinggi
2              0     laki-laki          46.863358            normal
3              0     laki-laki          47.508026            normal
4              0     laki-laki          42.743494  severely stunted
5              0     laki-laki          44.257719           stunted
6              0     laki-laki          59.572523            tinggi
7              0     laki-laki          42.701796  severely stunted
8              0     laki-laki          45.251779           stunted
9              0     laki-laki          57.201961            tinggi
10             0     laki-laki          51.353858            normal
11             0     laki-laki          53.049911            normal
12             0     laki-laki          43.544872  severely stunted
13             0     laki-laki          46.252632            normal
14             0     laki-laki          43.676035  severely stunted

In [8]:
# Melakukan encoding pada kolom Jenis Kelamin dan Status Gizi
label_encoder = LabelEncoder()
data['Jenis Kelamin'] = label_encoder.fit_transform(data['Jenis Kelamin'])
data['Status Gizi'] = label_encoder.fit_transform(data['Status Gizi'])

In [9]:
data.head(15)

Umur (bulan)  Jenis Kelamin  Tinggi Badan (cm)  Status Gizi
0              0              0          44.591973            2
1              0              0          56.705203            3
2              0              0          46.863358            0
3              0              0          47.508026            0
4              0              0          42.743494            1
5              0              0          44.257719            2
6              0              0          59.572523            3
7              0              0          42.701796            1
8              0              0          45.251779            2
9              0              0          57.201961            3
10             0              0          51.353858            0
11             0              0          53.049911            0
12             0              0          43.544872            1
13             0              0          46.252632            0
14             0              0          43.676035            1

In [10]:
# Memisahkan fitur dan target
X = data.drop('Status Gizi', axis=1)
y = data['Status Gizi']

In [11]:
# Membagi data menjadi data pelatihan dan data pengujian (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Membuat model Sequential dengan L2 regularization
from tensorflow.keras.regularizers import l2

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Melatih model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2, callbacks=[early_stopping])

Epoch 1/50
2420/2420 - 7s - loss: 1.4520 - accuracy: 0.5470 - val_loss: 1.0968 - val_accuracy: 0.5691 - 7s/epoch - 3ms/step
Epoch 2/50
2420/2420 - 7s - loss: 0.9369 - accuracy: 0.6256 - val_loss: 0.6144 - val_accuracy: 0.7806 - 7s/epoch - 3ms/step
Epoch 3/50
2420/2420 - 5s - loss: 0.6419 - accuracy: 0.7459 - val_loss: 0.4438 - val_accuracy: 0.8450 - 5s/epoch - 2ms/step
Epoch 4/50
2420/2420 - 5s - loss: 0.5467 - accuracy: 0.7838 - val_loss: 0.4420 - val_accuracy: 0.8235 - 5s/epoch - 2ms/step
Epoch 5/50
2420/2420 - 8s - loss: 0.4938 - accuracy: 0.8060 - val_loss: 0.3683 - val_accuracy: 0.8774 - 8s/epoch - 3ms/step
Epoch 6/50
2420/2420 - 8s - loss: 0.4569 - accuracy: 0.8231 - val_loss: 0.3914 - val_accuracy: 0.8222 - 8s/epoch - 3ms/step
Epoch 7/50
2420/2420 - 9s - loss: 0.4384 - accuracy: 0.8303 - val_loss: 0.4093 - val_accuracy: 0.8442 - 9s/epoch - 4ms/step
Epoch 8/50
2420/2420 - 5s - loss: 0.4145 - accuracy: 0.8408 - val_loss: 0.3966 - val_accuracy: 0.8237 - 5s/epoch - 2ms/step
Epoch 9/

In [14]:
from tensorflow.keras.models import load_model
import joblib
# Simpan LabelEncoder ke file
joblib.dump(label_encoder, 'label_encoder.pkl')
# Load the label encoder
label_encoder = joblib.load('label_encoder.pkl')

# Define the function to make predictions
def predict_status_gizi(umur, jenis_kelamin, tinggi_badan):
    # Preprocess the input
    if jenis_kelamin.lower() == 'laki-laki':
        jenis_kelamin_encoded = 1
    else:
        jenis_kelamin_encoded = 0

    input_data = np.array([[umur, jenis_kelamin_encoded, tinggi_badan]])

    # Make predictions
    predictions = model.predict(input_data)
    predicted_class = np.argmax(predictions, axis=1)

    # Decode the predicted class
    status_gizi = label_encoder.inverse_transform(predicted_class)

    return status_gizi[0]

In [34]:
# Example usage
umur = 0  # Umur dalam bulan
jenis_kelamin = 'Laki-laki'  # Jenis kelamin
tinggi_badan =   58.5# Tinggi badan dalam cm

predicted_status_gizi = predict_status_gizi(umur,jenis_kelamin,tinggi_badan)
print(f"Status Gizi: {predicted_status_gizi}")

1/1 [==============================] - 0s 29ms/step
Status Gizi: tinggi


In [16]:
# # Evaluasi model
# loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
# print("Akurasi model:", accuracy)

# # Melakukan prediksi menggunakan data pengujian
# y_pred = np.argmax(model.predict(X_test), axis=-1)

# # Mencetak laporan klasifikasi
# print("\nLaporan Klasifikasi:")
# print(classification_report(y_test, y_pred))

In [17]:
# # Konversi model ke format TensorFlow Lite
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Menyimpan model TensorFlow Lite ke direktori /content/sample_data
# tflite_model_path = '/content/sample_data/model_status_gizi_balita.tflite'
# with open(tflite_model_path, 'wb') as f:
#     f.write(tflite_model)

# print(f"Model telah disimpan sebagai '{tflite_model_path}'")

In [18]:
# # Load the TFLite model and allocate tensors
# tflite_model_path = '/content/sample_data/model_status_gizi_balita.tflite'
# interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
# interpreter.allocate_tensors()

# # Get input and output tensors
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Function to run inference on a single batch of data
# def predict_tflite(interpreter, X):
#     input_index = input_details[0]['index']
#     output_index = output_details[0]['index']

#     predictions = []
#     for i in range(X.shape[0]):
#         input_data = np.array(X[i], dtype=np.float32)
#         interpreter.set_tensor(input_index, [input_data])
#         interpreter.invoke()
#         output_data = interpreter.get_tensor(output_index)
#         predictions.append(output_data[0])

#     return np.array(predictions)

# # Run predictions on X_test
# X_test = X_test.values  # Ensure X_test is a numpy array
# y_pred_proba = predict_tflite(interpreter, X_test)
# y_pred = np.argmax(y_pred_proba, axis=-1)

# # Evaluate the predictions
# print("\nLaporan Klasifikasi:")
# print(classification_report(y_test, y_pred))

In [37]:
import os
from tensorflow.python.saved_model.save import save

save_dir = os.path.join('/tmp/','initgoogle_syslog_dir.0/saved_model.h5')
model.save_weights(save_dir)

In [38]:
!pip3 install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 895.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
   

In [39]:
!cd /tmp/initgoogle_syslog_dir.0/
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model /tmp/initgoogle_syslog_dir.0/saved_model.h5 /tmp/initgoogle_syslog_dir.0/

2024-06-18 15:52:00.982549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [41]:
!zip -r /tmp/initgoogle_syslog_dir.0/modeldata-lainnya.zip /tmp/initgoogle_syslog_dir.0/

  adding: tmp/initgoogle_syslog_dir.0/ (stored 0%)
  adding: tmp/initgoogle_syslog_dir.0/model.json (deflated 62%)
  adding: tmp/initgoogle_syslog_dir.0/group1-shard1of1.bin (deflated 5%)
  adding: tmp/initgoogle_syslog_dir.0/unknown (stored 0%)
  adding: tmp/initgoogle_syslog_dir.0/saved_model.h5 (deflated 48%)
